In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('/content/test.csv',header=None)

In [5]:
df

,0,1,2
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399988,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399989,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399990,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399991,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [6]:
df.rename({0:'target',1:'heading',2:'review'},axis=1,inplace=True)

In [7]:
df

,target,heading,review
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399988,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399989,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399990,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399991,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [8]:
df.isnull().sum()

target      0
heading    10
review      0
dtype: int64

In [9]:
df.drop('heading',axis=1,inplace=True)

In [10]:
df

,target,review
0,2,My lovely Pat has one of the GREAT voices of h...
1,2,Despite the fact that I have only played a sma...
2,1,I bought this charger in Jul 2003 and it worke...
3,2,Check out Maha Energy's website. Their Powerex...
4,2,Reviewed quite a bit of the combo players and ...
...,...,...
399988,1,We bought this Thomas for our son who is a hug...
399989,1,My son recieved this as a birthday gift 2 mont...
399990,1,"I bought this toy for my son who loves the ""Th..."
399991,2,This is a compilation of a wide range of Mitfo...


In [11]:
df['target'].value_counts()


1    199997
2    199996
Name: target, dtype: int64

In [12]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [13]:
df['review']=df['review'].apply(cleantext)

In [14]:
x = df['review']
y = df['target']

In [15]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=1)

In [16]:
sentlen = []

for sent in df["review"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen
df.head()

,target,review,SentLen
0,2,lovely pat one great voice generation listened...,56
1,2,despite fact played small portion game music h...,66
2,1,bought charger jul worked ok design nice conve...,27
3,2,check maha energy website powerex charger work...,17
4,2,reviewed quite bit combo player hesitant due u...,36


In [17]:
max(sentlen)

156

In [18]:
max_len = np.quantile(sentlen, 0.95)

In [19]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(X_train)
tok.index_word

{1: 'book',
 2: 'one',
 3: 'like',
 4: 'would',
 5: 'good',
 6: 'great',
 7: 'time',
 8: 'get',
 9: 'read',
 10: 'movie',
 11: 'really',
 12: 'work',
 13: 'well',
 14: 'even',
 15: 'much',
 16: 'first',
 17: 'love',
 18: 'make',
 19: 'could',
 20: 'product',
 21: 'year',
 22: 'cd',
 23: 'buy',
 24: 'story',
 25: 'use',
 26: 'album',
 27: 'also',
 28: 'better',
 29: 'bought',
 30: 'song',
 31: 'way',
 32: 'thing',
 33: 'think',
 34: 'new',
 35: 'many',
 36: 'know',
 37: 'best',
 38: 'little',
 39: 'back',
 40: 'want',
 41: 'got',
 42: 'go',
 43: 'never',
 44: 'two',
 45: 'say',
 46: 'still',
 47: 'music',
 48: 'see',
 49: 'find',
 50: 'money',
 51: 'people',
 52: 'character',
 53: 'game',
 54: 'made',
 55: 'day',
 56: 'recommend',
 57: 'look',
 58: 'old',
 59: 'sound',
 60: 'used',
 61: 'give',
 62: 'dvd',
 63: 'review',
 64: 'found',
 65: 'life',
 66: 'lot',
 67: 'ever',
 68: 'take',
 69: 'bad',
 70: 'put',
 71: 'need',
 72: 'reading',
 73: 'come',
 74: 'quality',
 75: 'film',
 76: 'an

In [20]:
vocab_len = len(tok.index_word)
vocab_len

170164

In [21]:
seqtrain = tok.texts_to_sequences(X_train) #step1
seqtrain

[[16,
  7558,
  3420,
  171,
  9,
  5319,
  1,
  4,
  9,
  76,
  171,
  678,
  5287,
  35,
  3380,
  939,
  337,
  3,
  6573,
  8,
  1704,
  217,
  2,
  77,
  88,
  203],
 [780,
  12,
  11,
  13,
  66,
  2036,
  780,
  1190,
  29,
  198,
  2746,
  5343,
  1401,
  29,
  780,
  1401,
  143,
  780,
  1422,
  2247,
  839,
  419,
  198,
  2746,
  2,
  975,
  93,
  7,
  557,
  74,
  5,
  87,
  77,
  1474,
  3811],
 [33, 9, 5, 1, 3936, 1960, 94, 303, 3061, 93, 7, 223],
 [172,
  21407,
  21408,
  436,
  2323,
  41,
  174,
  2,
  268,
  1643,
  263,
  954,
  222,
  409,
  670,
  1643,
  338,
  837,
  1065,
  409,
  7],
 [73,
  135,
  14343,
  2,
  37,
  23819,
  1,
  170,
  244,
  1371,
  13478,
  119,
  2606,
  1006,
  229,
  105,
  77,
  18,
  1420,
  22159,
  35,
  845,
  276,
  3,
  7934,
  207,
  6241,
  2817,
  5430,
  1041,
  40,
  10334,
  11421],
 [215,
  616,
  40,
  2509,
  774,
  567,
  330,
  11422,
  294,
  330,
  68,
  49,
  77,
  70,
  330,
  789,
  330],
 [3605, 3218, 188, 229,

In [22]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[   0,    0,    0, ...,   77,   88,  203],
       [   0,    0,    0, ...,   77, 1474, 3811],
       [   0,    0,    0, ...,   93,    7,  223],
       ...,
       [   0,    0,    0, ..., 1539, 3062,  815],
       [   0,    0,    0, ...,  888,  422,  327],
       [   0,    0,    0, ...,  167,   42,  864]], dtype=int32)

In [23]:
seqtest = tok.texts_to_sequences(X_test)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [24]:
max_len

75.0

In [25]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,50, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy",metrics=['accuracy'])

rnn.fit(seqmattrain, y_train, batch_size=50, epochs=10)

ypred = rnn.predict(seqmattest)

ypred = ypred>0.5

Epoch 1/10
5600/5600 [==============================] - 762s 135ms/step - loss: -3999.4565 - accuracy: 0.5007
Epoch 2/10
5600/5600 [==============================] - 683s 122ms/step - loss: -22656.1797 - accuracy: 0.5011
Epoch 3/10
5600/5600 [==============================] - 680s 121ms/step - loss: -54649.0625 - accuracy: 0.5011
Epoch 4/10
5600/5600 [==============================] - 669s 119ms/step - loss: -99318.7969 - accuracy: 0.5011
Epoch 5/10
5600/5600 [==============================] - 656s 117ms/step - loss: -156656.6875 - accuracy: 0.5011
Epoch 6/10
5600/5600 [==============================] - 653s 117ms/step - loss: -226257.0156 - accuracy: 0.5011
Epoch 7/10
5600/5600 [==============================] - 654s 117ms/step - loss: -308375.2812 - accuracy: 0.5011
Epoch 8/10
5600/5600 [==============================] - 656s 117ms/step - loss: -402830.7188 - accuracy: 0.5011
Epoch 9/10
5600/5600 [==============================] - 651s 116ms/step - loss: -509544.7188 - accuracy: 0.50

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test,ypred))

              precision    recall  f1-score   support

           1       0.50      1.00      0.66     59700
           2       0.00      0.00      0.00     60298

    accuracy                           0.50    119998
   macro avg       0.25      0.50      0.33    119998
weighted avg       0.25      0.50      0.33    119998

